In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re # for regex
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score
import pickle


In [2]:
data = pd.read_csv("IMDB-Dataset.csv")

In [3]:
data.shape

(50000, 2)

In [4]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [6]:
data.sentiment.value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [7]:
data.sentiment.replace('positive',1,inplace = True)
data.sentiment.replace('negative',0,inplace = True)
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [8]:
data.review[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [9]:
def clean(text):
    cleaned = re.compile(r'<.*?>')
    return re.sub(cleaned,'',text)

data.review = data.review.apply(clean)
data.review[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [10]:
def is_special(text):
    rem = ''
    for i in text :
        if i.isalnum():
            rem = rem + i
        else:
            rem = rem + ' '
    return rem

data.review = data.review.apply(is_special)
data.review [0]

'One of the other reviewers has mentioned that after watching just 1 Oz episode you ll be hooked  They are right  as this is exactly what happened with me The first thing that struck me about Oz was its brutality and unflinching scenes of violence  which set in right from the word GO  Trust me  this is not a show for the faint hearted or timid  This show pulls no punches with regards to drugs  sex or violence  Its is hardcore  in the classic use of the word It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary  It focuses mainly on Emerald City  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  Em City is home to many  Aryans  Muslims  gangstas  Latinos  Christians  Italians  Irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [11]:
def to_lower(text):
    return text.lower()
data.review = data.review.apply(to_lower)
data.review [0]
    

'one of the other reviewers has mentioned that after watching just 1 oz episode you ll be hooked  they are right  as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence  which set in right from the word go  trust me  this is not a show for the faint hearted or timid  this show pulls no punches with regards to drugs  sex or violence  its is hardcore  in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary  it focuses mainly on emerald city  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  em city is home to many  aryans  muslims  gangstas  latinos  christians  italians  irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wo

In [12]:
# Stop words removal for example a, the, and , or , of , on , this , we , were, is, not 

def rem_stopwords (text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize (text)
    
    return [w for w in words if w not in stop_words]

data.review = data.review.apply (rem_stopwords)
data.review [0]


['one',
 'reviewers',
 'mentioned',
 'watching',
 '1',
 'oz',
 'episode',
 'hooked',
 'right',
 'exactly',
 'happened',
 'first',
 'thing',
 'struck',
 'oz',
 'brutality',
 'unflinching',
 'scenes',
 'violence',
 'set',
 'right',
 'word',
 'go',
 'trust',
 'show',
 'faint',
 'hearted',
 'timid',
 'show',
 'pulls',
 'punches',
 'regards',
 'drugs',
 'sex',
 'violence',
 'hardcore',
 'classic',
 'use',
 'word',
 'called',
 'oz',
 'nickname',
 'given',
 'oswald',
 'maximum',
 'security',
 'state',
 'penitentary',
 'focuses',
 'mainly',
 'emerald',
 'city',
 'experimental',
 'section',
 'prison',
 'cells',
 'glass',
 'fronts',
 'face',
 'inwards',
 'privacy',
 'high',
 'agenda',
 'em',
 'city',
 'home',
 'many',
 'aryans',
 'muslims',
 'gangstas',
 'latinos',
 'christians',
 'italians',
 'irish',
 'scuffles',
 'death',
 'stares',
 'dodgy',
 'dealings',
 'shady',
 'agreements',
 'never',
 'far',
 'away',
 'would',
 'say',
 'main',
 'appeal',
 'show',
 'due',
 'fact',
 'goes',
 'shows',
 'da

In [13]:
def stem_txt (text):
    ss = SnowballStemmer ('english')
    return " ".join([ss.stem(w) for w in text])

data.review = data.review.apply(stem_txt)
data.review [0]

'one review mention watch 1 oz episod hook right exact happen first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word call oz nicknam given oswald maximum secur state penitentari focus main emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far away would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that get touch darker side'

In [14]:
data.head()

,review,sentiment
0,one review mention watch 1 oz episod hook righ...,1
1,wonder littl product film techniqu unassum old...,1
2,thought wonder way spend time hot summer weeke...,1
3,basic famili littl boy jake think zombi closet...,0
4,petter mattei love time money visual stun film...,1


In [15]:
X = np.array(data.iloc[:,0].values)
y = np.array(data.sentiment.values)
cv = CountVectorizer(max_features = 12500)
X = cv.fit_transform(data.review).toarray()
print("X.shape = ",X.shape)
print("y.shape = ",y.shape)

X.shape =  (50000, 12500)
y.shape =  (50000,)


In [16]:
trainx,testx,trainy,testy = train_test_split (X,y,test_size = 0.2, random_state = 42, shuffle = True, stratify = y)
print ("Train shapes : X = {}, y = {}".format(trainx.shape,trainy.shape))
print ("Test shapes : X = {}, y = {}".format(testx.shape,testy.shape))

Train shapes : X = (40000, 12500), y = (40000,)
Test shapes : X = (10000, 12500), y = (10000,)


In [17]:
gnb = GaussianNB ()
gnb.fit(trainx,trainy)

GaussianNB()

In [18]:
ypg = gnb.predict (testx)
print ("Gaussian =",accuracy_score(testy,ypg))

Gaussian = 0.6782


In [19]:
mnb= MultinomialNB(alpha=1.0,fit_prior=True)
mnb.fit(trainx,trainy)

MultinomialNB()

In [20]:
ypm = mnb.predict (testx)
print("Multinomial",accuracy_score(testy,ypm))

Multinomial 0.8479


In [21]:
bnb = BernoulliNB (alpha =1.0, fit_prior = True)
bnb.fit(trainx, trainy)

BernoulliNB()

In [22]:
ypm = bnb.predict (testx)
print("BernoulliNB:",accuracy_score(testy,ypm))

BernoulliNB: 0.8496


In [23]:
# Save model and vectorizer
pickle.dump(bnb,open ('./moviereviews.pkl','wb'))
pickle.dump(cv,open ('./vectorizer.pkl','wb'))

In [24]:
# Load both
bnb_loaded = pickle.load(open('./moviereviews.pkl', 'rb'))
cv_loaded = pickle.load(open('./vectorizer.pkl', 'rb'))

# New reviews
new_reviews = [
    "The movie was absolutely amazing and thrilling!",
    "The movie was great."
]

# Apply same preprocessing as training
processed_reviews = []
for review in new_reviews:
    review = is_special(review)
    review = rem_stopwords(review)
    review = stem_txt(review)
    processed_reviews.append(review)

# Transform and predict
new_reviews_transformed = cv_loaded.transform(processed_reviews).toarray()
predictions = bnb_loaded.predict(new_reviews_transformed)

print(predictions)  # Should now give more accurate results

[1 0]


In [25]:
import pickle
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB

def full_preprocess(text):
    text = is_special(text)
    text = rem_stopwords(text)
    text = stem_txt(text)
    return text

# Build the pipeline
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(max_features=12500, preprocessor=full_preprocess)),
    ('classifier', BernoulliNB())
])

# Train the pipeline
pipeline.fit(data.review, data.sentiment)

# Save the whole pipeline
pickle.dump(pipeline, open('./moviereviews_pipeline.pkl', 'wb'))

In [26]:
import streamlit as st
import pickle
import pandas as pd
# Load saved pipeline
pipeline = pickle.load (open('./moviereviews_pipeline.pkl','rb'))



#Title

st.title ("🎬 Review Sentiment Analyzer")
          
st.write("Enter a review or upload a csv file with reviews to predict sentiment")
          
# -- Single review input --
st.subheader ("Single Review Prediction")
          
review = st.text_area ("Enter your review here:")
          

if st.button("Predict Sentiment"):
    if review.strip() != "":
        prediction = pipeline.predict([review])[0]
        proba = pipeline.predict_proba([review])[0]

        sentiment = "Positive" if prediction == 1 else "Negative"
        emoji = "😊" if prediction == 1 else "😞"
        color = "green" if prediction == 1 else "red"


        st.markdown(f"**Predicted Sentiment:** <span style='color:{color}'>{sentiment}</span>", unsafe_allow_html=True)
        st.markdown(f"**Confidence:** Positive = {proba[1]:.2f}, Negative = {proba[0]:.2f}")
    else:
        st.warning("Please enter a review text!")
          




2025-08-15 00:03:56.422 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 00:03:57.584 
  command:

    streamlit run C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-08-15 00:03:57.585 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 00:03:57.586 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 00:03:57.587 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 00:03:57.590 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 00:03:57.591 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 00:03:57.592 Thread 'MainThread': mi

In [27]:
print("Great movie →", pipeline.predict(["The movie was great"]))
print("Terrible movie →", pipeline.predict(["The movie was terrible"]))
print("Awesome movie →", pipeline.predict(["Awesome movie"]))
print("Awful movie →", pipeline.predict(["Awful movie"]))

# Check dataset balance
print("Class balance in dataset:")
print(data.sentiment.value_counts())


Great movie → [0]
Terrible movie → [0]
Awesome movie → [0]
Awful movie → [0]
Class balance in dataset:
sentiment
1    25000
0    25000
Name: count, dtype: int64
